In [1]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar

dataset_name = "super_glue"  # The MMLU dataset name in Hugging Face Datasets
dataset = load_dataset(dataset_name, name='multirc')#["train"]
# print(dataset)

labels = dataset["train"].features["label"]
# def create_label_str(batch):
#     return {"label_str": labels.int2str(batch["label"])}
def create_label_str(example):
    return {"label_str": "True" if example["label"]== 1 else "False"}
# print(labels)


Generating train split:   0%|          | 0/27243 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9693 [00:00<?, ? examples/s]

In [2]:
# data_train = dataset["train"].map(create_label_str)   
# data_test = dataset["validation"].map(create_label_str)   
# # print(d1)
# # dd = d1.select(range(5)).shuffle()

d1 = dataset["train"].map(create_label_str)   
data_test = dataset["validation"].map(create_label_str)   

# d_all = concatenate_datasets([d1,d2])
train_size = 1000
data_train = d1.select(range(train_size))
# data_test = d_all.select(range(train_size,d_all.num_rows))

print(data_train.num_rows, data_test.num_rows)

# prepare examples
example_prompt = []
ex_inds = torch.randint(0, 80,(6,))
print(ex_inds)
for i, ex in enumerate(ex_inds):
    dd = data_train.filter(lambda example: example['idx']['question']==ex)
    idx = 0
    prompt = "Paragraph: " + dd[idx]['paragraph'] + '\n'
    for idx in [0]:
        prompt += "Question: " + dd[idx]['question'] + '\n'
        prompt += "Response: " + dd[idx]['answer'] + '\n'
        prompt += "Answer:" + dd[idx]['label_str'] 
        # prompt += dd[idx]['label_str'] #+ '\n\n'
    example_prompt.append(prompt)

Map: 100%|██████████| 4848/4848 [00:00<00:00, 13973.75 examples/s]


1000 4848
tensor([11, 37, 43, 73, 58, 45])


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 48973.13 examples/s]


In [3]:
disable_progress_bar()
num_qs_all = 0
step = 100
while len(data_test.filter(lambda example: example['idx']['question']==num_qs_all))>0:
    num_qs_all += 1
    if num_qs_all % step == 0:
        print(f"{num_qs_all//step}", end='\r')
print(num_qs_all)
enable_progress_bar()

953


In [20]:

def json_writer(JSON_PATH, num_qs, n_shot_example,seed=10,space=False):
    qlist = {}
    
    with open(JSON_PATH, 'w') as json_file:

        step = 100
        # ind = 0
        # while len(data_test.filter(lambda example: example['idx']['question']==ind))>0:
        for ind in range(num_qs):
            ind += 1
            if ind % step == 0:
                print(f"{ind//step}", end='\r')

            idx_ex_shuffle = torch.randperm(5)
            init_prompt = "Based on the paragraph, determine whether the candidate answer is True or False."
            for i in range(n_shot_example):
                init_prompt +=  example_prompt[idx_ex_shuffle[i]] + '\n\n'

            dd = data_test.filter(lambda example: example['idx']['question']==ind)

            init_prompt += "Paragraph: " + dd[0]['paragraph'] + '\n'
            for idx_q in range(dd.num_rows):
                prompt = init_prompt
                prompt += "Question: " + dd[idx_q]['question'] + '\n'
                prompt += "Candidate answer: " + dd[idx_q]['answer'] + '\n'
                prompt += "Answer:"
                
                qlist['question'] = ind
                qlist['input'] = prompt
                qlist['target_str'] = dd[idx_q]['label_str']
                qlist['target'] = dd[idx_q]['label']

                 # Serializing json
                json_file.write(json.dumps(qlist) + '\n')
        # print("Done!")

In [21]:
disable_progress_bar()

# print(example_idx)
n_example = 1000#data_test.num_rows
n_shot_vals = [2]
seed = 123
for n_shot in range(6):
# for n_shot in n_shot_vals:
    print(f"{n_shot}-shot example is being generated...")
    fname_json = f"multirc-data/{n_shot}_shot_examples.json"
    # fname_json = f"boolq-data/{n_shot}_shot_examples_small.json"
    # JSON_eval = 'sample_eval.json'
    json_writer(fname_json, n_example, n_shot, seed=seed, space= False)
    # json_writer(JSON_eval,example_idx[(n_shot_example+1)*n_test_ex:])

enable_progress_bar()


0-shot example is being generated...


KeyboardInterrupt: 

In [9]:
# fname_json = f"mmlu-data/ex_no_space.json"
n_shot = 0
d1 = load_dataset("json", data_files={
        'dev' : f"multirc-data/{n_shot}_shot_examples.json"
    })
# print(d1['dev'])
# print(d1['dev'].filter(lambda example: example["subject"] == subject))
# d2 = d1['dev'].filter(lambda example: example["subject"] == subject)
# # print(d2['input'][0][-1])
# # print("---------")
# print(d1['dev'][2]['input'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [10]:
ex = 0
dd = d1['dev'].filter(lambda example: example['question']==ex)
print(dd)
for i in range(dd.num_rows):
    print(dd['input'][i])
    print(f"{''.join(['_']*100)}")


Filter:   0%|          | 0/4848 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'input', 'target_str', 'target'],
    num_rows: 4
})
Paragraph: What causes a change in motion? The application of a force. Any time an object changes motion, a force has been applied. In what ways can this happen? Force can cause an object at rest to start moving. Forces can cause objects to speed up or slow down. Forces can cause a moving object to stop. Forces can also cause a change in direction. In short, forces cause changes in motion. The moving object may change its speed, its direction, or both. We know that changes in motion require a force. We know that the size of the force determines the change in motion. How much an objects motion changes when a force is applied depends on two things. It depends on the strength of the force. It also depends on the objects mass. Think about some simple tasks you may regularly do. You may pick up a baseball. This requires only a very small force. 
Question: Would the mass of a baseball affect how much fo

In [ ]:
from datasets import DatasetDict, Dataset
from collections import defaultdict

fname_dict = {}
for n_shot in range(6):
    fname_dict[f"{n_shot}_shot"] = f"multirc-data/{n_shot}_shot_examples.json"
# data_all = defaultdict(DatasetDict)
d_upload = load_dataset("json", data_files=fname_dict)
    # data_all[f"n_shot = {n_shot}"] = d1['dev']


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 2016.65it/s]
Generating 0_shot split: 50 examples [00:00, 10128.72 examples/s]
Generating 1_shot split: 50 examples [00:00, 11395.09 examples/s]
Generating 2_shot split: 50 examples [00:00, 15454.33 examples/s]
Generating 3_shot split: 50 examples [00:00, 6900.12 examples/s]
Generating 4_shot split: 50 examples [00:00, 6682.87 examples/s]
Generating 5_shot split: 50 examples [00:00, 6888.78 examples/s]


In [11]:
# print(Dataset.from_dict(data_all))
# Dataset.from_dict(data_all).push_to_hub("hassansh/boolq_n_shot")
d_upload.push_to_hub("hassansh/multirc_n_shot")


NameError: name 'd_upload' is not defined

In [21]:
# print(d_upload['5_shot'][2]['input'])
num_shot = 0
d_test = load_dataset("hassansh/multirc_n_shot",name = f"{num_shot}_shot")['test']
print(d_test)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'input', 'target_str', 'target'],
    num_rows: 4848
})


In [22]:
ex = 0
dd = d_test.filter(lambda example: example['question']==ex)
print(dd)
for i in range(dd.num_rows):
    print(dd['input'][i])
    print(f"{''.join(['_']*100)}")


Filter:   0%|          | 0/4848 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'input', 'target_str', 'target'],
    num_rows: 4
})
Paragraph: What causes a change in motion? The application of a force. Any time an object changes motion, a force has been applied. In what ways can this happen? Force can cause an object at rest to start moving. Forces can cause objects to speed up or slow down. Forces can cause a moving object to stop. Forces can also cause a change in direction. In short, forces cause changes in motion. The moving object may change its speed, its direction, or both. We know that changes in motion require a force. We know that the size of the force determines the change in motion. How much an objects motion changes when a force is applied depends on two things. It depends on the strength of the force. It also depends on the objects mass. Think about some simple tasks you may regularly do. You may pick up a baseball. This requires only a very small force. 
Question: Would the mass of a baseball affect how much fo

In [ ]:
disable_progress_bar()

num_paragraphs = 0
for i in range(d_test.num_rows):
    if d_test.filter(lambda example: example['question']==i):
        num_paragraphs += 1
    else:
        break

enable_progress_bar()


In [ ]:
print(num_paragraphs)

10
